In [ ]:
import cv2
import numpy as np
from PIL import Image
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
import math
from sklearn import preprocessing

In [ ]:
camera_records = glob.glob('images/camera_*.yuv')

In [ ]:
def write_yuv_to_jpg(yuv_path, folder, w, h):
    # Read entire file into YUV
    YUV = np.fromfile(yuv_path,dtype='uint8')
    Y = YUV[0:w*h].reshape(h,w)
    im = Image.fromarray(Y)
    file_name = yuv_path.split('/')[1].split('.yuv')[0]
    im.save(folder + '/' + file_name + '.jpg')
    

In [ ]:
def read_yuv_to_arr(yuv_path, w, h):
    YUV = np.fromfile(yuv_path,dtype='uint8')
    Y = YUV[0:w*h].reshape(h,w)
    return Y

In [ ]:
# # one time converstion from yuv to jpg
# for record in camera_records:
#     write_yuv_to_jpg(record, 'images', 256, 154)

In [ ]:
control_records = glob.glob('images/control_*.txt')

In [ ]:
tmp_records = [record.split('_', 1)[1] for record in control_records]
time_stamps = [record.split('.txt', 1)[0] for record in tmp_records]
control_time_stamps = list(map(float, time_stamps))

In [ ]:
def find_nearest(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return array[idx-1]
    else:
        return array[idx]

In [ ]:
sorted_control_time_stamps = np.sort(control_time_stamps)

In [ ]:
steers = []
accs = []
imgs = []
for record in camera_records:
    camera_time_stamp = float(record.split('_', 1)[1].split('.yuv', 1)[0])
    control_time_stamp = find_nearest(sorted_control_time_stamps, camera_time_stamp)
    path = 'images/control_' + str(control_time_stamp) + '.txt'
    with open(path) as control_file:
        control_str = control_file.readline()
        steer = int(control_str.split('x')[0].split('s')[1])
        acc = int(control_str.split('x')[1].split('a')[1])
        steers = np.append(steers, steer)
        accs = np.append(accs, acc)
        imgs = np.append(imgs, record.split('/', 1)[1].split('.yuv')[0] + '.jpg')

In [ ]:
df = pd.DataFrame()
df['file_name'] = imgs
df['throttle'] = accs
df['angle'] = steers

In [ ]:
df.shape

In [ ]:
# only using throttle over or equal 1500 to filter out unexpected stoppings along the track
df = df.loc[df['throttle'] >= 1500]


In [ ]:
df.shape

In [ ]:
# normalize the steering and angle
df['throttle'] = (df['throttle'] - 1400) / (1800 - 1400)
df['angle'] = (df['angle'] - 60) / (140 - 60)

In [ ]:
df_train, df_val = train_test_split(df, test_size=0.2)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

image_loader = ImageDataGenerator()

train_generator = image_loader.flow_from_dataframe(dataframe=df_train, directory="images", x_col='file_name', y_col=['throttle', 'angle'], target_size=(154, 256), color_mode='grayscale', class_mode='other', batch_size=32)

val_generator = image_loader.flow_from_dataframe(dataframe=df_val, directory='images', x_col='file_name', y_col=['throttle', 'angle'], target_size=(154, 256), color_mode='grayscale', class_mode='other', batch_size=32)


In [ ]:
from keras import Sequential
from keras.layers import Input, Cropping2D, Dense
from keras.layers import Dropout
from keras.layers import Convolution2D, Flatten
from keras.layers import BatchNormalization
from keras import Model

In [ ]:
def build_CNN(num_outputs, input_shape=(154, 256, 1), roi_crop=(0, 0)):

    drop = 0.1
    
    img_in = Input(shape=input_shape, name='img_in')
    x = img_in
    x = Cropping2D(cropping=(roi_crop, (0,0)))(x) #trim pixels off top and bottom

    x = BatchNormalization()(x)
    x = Convolution2D(24, (5,5), strides=(2,2), activation='relu', name="conv2d_1")(x)
    x = Dropout(drop)(x)
    x = Convolution2D(32, (5,5), strides=(2,2), activation='relu', name="conv2d_2")(x)
    x = Dropout(drop)(x)
    x = Convolution2D(64, (5,5), strides=(2,2), activation='relu', name="conv2d_3")(x)
    x = Dropout(drop)(x)
    x = Convolution2D(64, (3,3), strides=(1,1), activation='relu', name="conv2d_4")(x)
    x = Dropout(drop)(x)
    x = Convolution2D(64, (3,3), strides=(1,1), activation='relu', name="conv2d_5")(x)
    x = Dropout(drop)(x)
    
    x = Flatten(name='flattened')(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(drop)(x)
    x = Dense(50, activation='relu')(x)
    x = Dropout(drop)(x)

    # regression model with linear activation
    outputs = Dense(num_outputs, activation='linear', name='n_outputs')(x)
        
    model = Model(inputs=[img_in], outputs=outputs)
    
    return model

In [ ]:
model = build_CNN(num_outputs = 2)

In [ ]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')


In [ ]:
history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=100,
                    validation_data=val_generator,
                    validation_steps=20,
                    epochs=5)

In [ ]:
model.save('models/model-cnn.h5')

In [ ]:
pyplot.title('Loss / Mean Squared Error')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
im_arr = read_yuv_to_arr('images/camera_1562107676.89651.yuv', 256, 154)
im_arr = np.reshape(im_arr,[1,154,256,1])

In [ ]:
im_arr.shape

In [ ]:
from keras.models import load_model
model = load_model('models/model.h5')

In [ ]:
arr = model.predict(im_arr)

In [ ]:
arr[0][1]